In [1]:
import pandas as pd

CSV_PATH = "groupAVR_chat_input.csv"  # השם המדויק של ה-CSV שלך
messages_df = pd.read_csv(CSV_PATH)
messages_df.head()

,msg_id,app_protocol,src_app,dst_app,message,timestamp
0,1,CHAT_APP,client_user,chat_server,Hi,0.000
1,2,CHAT_APP,chat_server,client_user,Hello! How can I help?,0.010
2,3,CHAT_APP,client_user,chat_server,My cat is stuck on a tree - please save her!,0.030
3,4,CHAT_APP,chat_server,client_user,Got it. Where are you located?,0.045
4,5,CHAT_APP,client_user,chat_server,Near the park on Herzl street.,0.070


In [2]:
import socket, struct, random, time, platform
from typing import Optional

def checksum(data: bytes) -> int:
    """
    חישוב checksum כמו ב-IP/TCP:
    - אם האורך אי-זוגי מוסיפים 0
    - מחברים מילים בגודל 16 ביט
    - "מקפלים" את הנשיאה
    - עושים NOT בסוף
    """
    if len(data) % 2:
        data += b'\0'
    res = sum(struct.unpack('!%dH' % (len(data) // 2), data))
    while res >> 16:
        res = (res & 0xFFFF) + (res >> 16)
    return ~res & 0xFFFF


def hexdump(data: bytes, width: int = 16):
    """
    הדפסה יפה של הבייטים – גם ב־hex וגם ב־ASCII.
    שימושי לבדיקה ויזואלית של ה־packet.
    """
    for i in range(0, len(data), width):
        chunk = data[i:i + width]
        hex_bytes = ' '.join(f'{b:02x}' for b in chunk)
        ascii_bytes = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        print(f"{i:04x}  {hex_bytes:<{width * 3}}  {ascii_bytes}")


In [3]:
def build_ip_header(src_ip: str, dst_ip: str, payload_len: int,
                    proto: int = socket.IPPROTO_TCP) -> bytes:
    """
    בונה כותרת IPv4 בסיסית:
    - גרסה 4
    - IHL = 5 (בלי options)
    - TTL = 64
    - חישוב checksum מעל כל הכותרת
    """
    version_ihl = (4 << 4) + 5  # 4 ביט גרסה, 4 ביט IHL
    tos = 0
    total_length = 20 + payload_len  # כותרת IP (20 בייט) + שאר המטען
    identification = random.randint(0, 65535)
    flags_fragment = 0
    ttl = 64
    header_checksum = 0

    src = socket.inet_aton(src_ip)
    dst = socket.inet_aton(dst_ip)

    # שלב ראשון – בלי checksum
    ip_header = struct.pack(
        '!BBHHHBBH4s4s',
        version_ihl,
        tos,
        total_length,
        identification,
        flags_fragment,
        ttl,
        proto,
        header_checksum,
        src,
        dst
    )

    # חישוב checksum
    chksum = checksum(ip_header)

    # בנייה מחדש עם checksum
    ip_header = struct.pack(
        '!BBHHHBBH4s4s',
        version_ihl,
        tos,
        total_length,
        identification,
        flags_fragment,
        ttl,
        proto,
        chksum,
        src,
        dst
    )

    return ip_header


In [4]:
def build_tcp_header(src_ip: str,
                     dst_ip: str,
                     src_port: int,
                     dst_port: int,
                     payload: bytes = b'',
                     seq: Optional[int] = None,
                     ack_seq: int = 0,
                     flags: int = 0x18,  # ברירת מחדל: PSH+ACK
                     window: int = 65535) -> bytes:
    """
    בונה כותרת TCP:
    - כולל חישוב checksum עם pseudo-header של IP
    - לא מטפלים פה ב־options, URG וכו'
    """
    if seq is None:
        seq = random.randint(0, 0xFFFFFFFF)

    doff_reserved = (5 << 4)  # Data offset = 5 (ללא options), 4 ביט רזרבה
    checksum_tcp = 0
    urg_ptr = 0

    # כותרת ראשונית בלי checksum
    tcp_header = struct.pack(
        '!HHLLBBHHH',
        src_port,
        dst_port,
        seq,
        ack_seq,
        doff_reserved,
        flags,
        window,
        checksum_tcp,
        urg_ptr
    )

    # pseudo-header ל-checksum
    placeholder = 0
    protocol = socket.IPPROTO_TCP
    tcp_length = len(tcp_header) + len(payload)

    pseudo_header = struct.pack(
        '!4s4sBBH',
        socket.inet_aton(src_ip),
        socket.inet_aton(dst_ip),
        placeholder,
        protocol,
        tcp_length
    )

    chksum = checksum(pseudo_header + tcp_header + payload)

    # בנייה מחדש עם checksum
    tcp_header = struct.pack(
        '!HHLLBBHHH',
        src_port,
        dst_port,
        seq,
        ack_seq,
        doff_reserved,
        flags,
        window,
        chksum,
        urg_ptr
    )

    return tcp_header


In [5]:
!pip install scapy


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import platform

IS_WINDOWS = (platform.system() == "Windows")

try:
    from scapy.all import IP as SCAPY_IP, TCP as SCAPY_TCP, Raw as SCAPY_Raw, send as scapy_send, get_if_list
    HAVE_SCAPY = True
except Exception as e:
    HAVE_SCAPY = False
    SCAPY_IMPORT_ERR = e

IS_WINDOWS, HAVE_SCAPY


(True, True)

In [27]:
class RawTcpTransport:
    """
    מחלקה שאחראית:
    - לקבל payload (bytes)
    - לבנות עבורו TCP header + IP header
    - לשגר החוצה (או דרך raw socket בלינוקס, או דרך Scapy ב-Windows)
    """
    def __init__(self, src_ip: str, dst_ip: str, src_port: int, dst_port: int, iface: Optional[str] = None):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
        self.iface = iface

        self.windows_fallback = IS_WINDOWS

        if not self.windows_fallback:
            # לינוקס / מק – משתמשים ב-raw socket
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)
        else:
            # Windows – משתמשים ב-Scapy
            if not HAVE_SCAPY:
                raise RuntimeError(
                    f"Windows detected but Scapy is not available: {SCAPY_IMPORT_ERR}.\n"
                    "Install with: pip install scapy. Ensure Npcap is installed with loopback support."
                )

    def encapsulate(self, data: bytes, flags: int = 0x18) -> bytes:
        """
        בונה packet שלם: IP + TCP + payload
        flags=0x18 זה PSH+ACK (דחוף + אושר)
        """
        tcp = build_tcp_header(self.src_ip, self.dst_ip, self.src_port, self.dst_port, data, flags=flags)
        ip  = build_ip_header(self.src_ip, self.dst_ip, len(tcp) + len(data))
        return ip + tcp + data

    def send(self, data: bytes, flags: int = 0x18):
        """
        שולח את ה-packet:
        - על לינוקס/מק: עם raw socket
        - על Windows: עם Scapy
        """
        if not self.windows_fallback:
            pkt = self.encapsulate(data, flags=flags)
            self.sock.sendto(pkt, (self.dst_ip, 0))
        else:
            scapy_pkt = SCAPY_IP(src=self.src_ip, dst=self.dst_ip) / \
                        SCAPY_TCP(sport=self.src_port, dport=self.dst_port, flags=flags) / \
                        SCAPY_Raw(data)
            chosen_iface = self.iface
            # אם היעד הוא loopback – ניתן לבחור Loopback
            if chosen_iface is None and self.dst_ip in ("127.0.0.1", "::1"):
                chosen_iface = "Npcap Loopback Adapter"
            scapy_send(scapy_pkt, verbose=False, iface=chosen_iface)


In [25]:
src_ip = "127.0.0.1"
dst_ip = "127.0.0.1"
src_port = random.randint(1024, 65535)  # פורט מקור אקראי
dst_port = 12345                        # פורט יעד קבוע לצורך הפרויקט

iface = None  # אפשר גם לשים כאן את שם ה-loopback, למשל "Npcap Loopback Adapter" אם צריך

transport = RawTcpTransport(src_ip, dst_ip, src_port, dst_port, iface=iface)

src_ip, dst_ip, src_port, dst_port


('127.0.0.1', '127.0.0.1', 21892, 12345)

In [23]:
test_payload = b"hello from notebook"
packet_bytes = transport.encapsulate(test_payload)
hexdump(packet_bytes[:64])  # מציגים רק את 64 הבייטים הראשונים


0000  45 00 00 3b 89 08 00 00 40 06 f3 b2 7f 00 00 01   E..;....@.......
0010  7f 00 00 01 17 c7 30 39 db a4 0a d7 00 00 00 00   ......09........
0020  50 18 ff ff 99 b4 00 00 68 65 6c 6c 6f 20 66 72   P.......hello fr
0030  6f 6d 20 6e 6f 74 65 62 6f 6f 6b                  om notebook


In [26]:
from scapy.all import IP, TCP, Raw, send


pkt = IP(src=src_ip, dst=dst_ip) / \
      TCP(sport=src_port, dport=dst_port, flags="PA") / \
      Raw(test_payload)

pkt.show()         # רק כדי לראות מה בנינו
send(pkt)          # שולח את הפקטה
print("packet sent!")


###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \
###[ TCP ]###
     sport     = 21892
     dport     = 12345
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = PA
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = []
###[ Raw ]###
        load      = b'hello from notebook'


Sent 1 packets.
packet sent!
